In [ ]:
# Install system dependencies for Box2D
!apt-get install -y swig

# Install Python libraries
!pip install gymnasium[box2d] stable-baselines3 shimmy moviepy matplotlib pandas

In [ ]:
import base64
from pathlib import Path
from IPython.display import HTML, display

def show_video(video_path):
    """Embeds an MP4 video directly in the Colab notebook."""
    video_path = Path(video_path)
    if not video_path.exists():
        print(f"Video not found at {video_path}")
        return

    video_content = video_path.read_bytes()
    b64 = base64.b64encode(video_content).decode()

    html_template = f'''
    <video width="640" height="480" controls>
      <source src="data:video/mp4;base64,{b64}" type="video/mp4">
    </video>
    '''
    display(HTML(html_template))

In [ ]:
import gymnasium as gym
import glob
from gymnasium.wrappers import RecordVideo

# Setup the environment
env = gym.make("BipedalWalker-v3", render_mode="rgb_array")
# Record every episode (trigger=lambda x: True)
env = RecordVideo(env, video_folder="./videos/test_run", name_prefix="test_run", episode_trigger=lambda x: True)

print("--- Running Video Test (Random Agent) ---")
env.reset()

for _ in range(200):  # Run for just 200 steps to generate a quick clip
    action = env.action_space.sample()  # Pick a random action
    _, _, terminated, truncated, _ = env.step(action)
    if terminated or truncated:
        env.reset()

env.close()
print("Test Run Complete.")

# Display the video
video_files = glob.glob("./videos/test_run/*.mp4")
if video_files:
    print(f"Found video: {video_files[0]}")
    show_video(video_files[0])
else:
    print("No video found. Check if ffmpeg/moviepy installed correctly.")

/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /content/videos/test_run folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


--- Running Video Test (Random Agent) ---
Test Run Complete.
Found video: ./videos/test_run/test_run-episode-3.mp4


In [ ]:
import gymnasium as gym
import torch.nn as nn
import numpy as np
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor

# Directories
LOG_DIR = "./logs/improved/"
VIDEO_DIR = "./videos/improved/"
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

# --- 1. Custom Reward Wrapper ---
class StabilityRewardWrapper(gym.Wrapper):
    def __init__(self, env, upright_bonus=0.1):
        super().__init__(env)
        self.upright_bonus = upright_bonus

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        # obs[0] is hull angle. We want it close to 0.
        stability = self.upright_bonus * np.exp(-10 * (obs[0] ** 2))
        return obs, reward + stability, terminated, truncated, info

def make_env():
    env = gym.make("BipedalWalker-v3", render_mode="rgb_array")
    # Wrap to record video every 50 episodes, length 2000 steps
    env = gym.wrappers.RecordVideo(
        env,
        video_folder=VIDEO_DIR,
        episode_trigger=lambda x: x % 50 == 0,
        name_prefix="bipedal-agent",
        video_length=2000
    )
    env = StabilityRewardWrapper(env)
    env = Monitor(env, LOG_DIR)
    return env

# --- 2. Environment Setup ---
# Create environment
env = DummyVecEnv([make_env])

# CRITICAL: Normalize observations (-3.14 to +5.0 is too wide for raw input)
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

# --- 3. The Bigger "Brain" (Policy) ---
# We need a larger network to process the 24 inputs (Lidar + Joints)
policy_kwargs = dict(
    activation_fn=nn.Tanh,
    net_arch=dict(pi=[256, 256], vf=[256, 256])
)

model = PPO(
    "MlpPolicy",
    env,
    policy_kwargs=policy_kwargs,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    ent_coef=0.001, # Small entropy to prevent "scooting" on knees
    verbose=1
)

# --- 4. Train ---
print("Starting Training (This may take 15-20 mins on Colab)...")
# Note: To truly "solve" it (300+ reward), you often need ~500k-1M steps.
# We run 200k here to show progress.
model.learn(total_timesteps=200_000)

# Save Model & Normalization Stats
model.save(f"{LOG_DIR}/final_model")
env.save(f"{LOG_DIR}/vec_normalize.pkl")
env.close()

print("Training Done!")

In [ ]:
import glob

# Find the latest video in the folder
video_files = glob.glob(f"{VIDEO_DIR}/*.mp4")
# Sort by modification time to get the newest one
video_files.sort(key=os.path.getmtime)

if video_files:
    latest_video = video_files[-1]
    print(f"Showing: {latest_video}")
    show_video(latest_video)
else:
    print("No videos found.")

Showing: ./videos/improved/bipedal-agent-episode-150.mp4


In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from gymnasium.wrappers import RecordVideo
import glob
import os

# 1. Define the Hardcore Environment
def make_hardcore():
    # hardcore=True turns on ladders, stumps, and pitfalls
    env = gym.make("BipedalWalker-v3", hardcore=True, render_mode="rgb_array")
    env = RecordVideo(
        env,
        video_folder="./videos/hardcore",
        name_prefix="hardcore_run",
        episode_trigger=lambda x: True # Record every single attempt
    )
    return env

# 2. Setup the environment
print("--- Setting up Hardcore Environment ---")
hardcore_env = DummyVecEnv([make_hardcore])

# 3. Load the "Memory" (Normalization Stats)
# The agent needs to know what "normal" data looks like.
# We load the stats from your previous training run.
stats_path = "./logs/improved/vec_normalize.pkl"
if os.path.exists(stats_path):
    print("Loading normalization stats...")
    hardcore_env = VecNormalize.load(stats_path, hardcore_env)
    # Important: Don't update stats during this test run, just use them.
    hardcore_env.training = False
    hardcore_env.norm_reward = False
else:
    print("Warning: Normalization stats not found. Agent might fail instantly.")

# 4. Load the Trained Brain
model_path = "./logs/improved/final_model.zip"
if os.path.exists(model_path):
    print("Loading trained model...")
    model = PPO.load(model_path, env=hardcore_env)
else:
    raise FileNotFoundError("Could not find the trained model! Did Cell 4 finish?")

# 5. Run the Agent
print("--- 🏃 STARTING HARDCORE RUN 🏃 ---")
obs = hardcore_env.reset()
done = False

while not done:
    # Predict the action
    action, _ = model.predict(obs, deterministic=True)

    # Step in the environment
    obs, reward, done, info = hardcore_env.step(action)

hardcore_env.close()
print("--- Run Complete ---")

# 6. Show the Video
video_files = glob.glob("./videos/hardcore/*.mp4")
if video_files:
    # Sort to show the newest video
    video_files.sort(key=os.path.getmtime)
    latest = video_files[-1]
    print(f"Showing video: {latest}")
    show_video(latest)
else:
    print("No video found. The agent might have failed before the first frame was saved.")

--- Setting up Hardcore Environment ---
Loading normalization stats...
Loading trained model...
--- 🏃 STARTING HARDCORE RUN 🏃 ---


/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /content/videos/hardcore folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


--- Run Complete ---
Showing video: ./videos/hardcore/hardcore_run-episode-1.mp4


In [ ]:
# ... (Imports from previous cells) ...

# 1. Load the Hardcore Env for TRAINING
def make_hardcore_train():
    env = gym.make("BipedalWalker-v3", hardcore=True, render_mode="rgb_array")
    env = Monitor(env, "./logs/hardcore_train/") # distinct log folder
    return env

print("--- Starting Curriculum Learning (Normal -> Hardcore) ---")

# Setup Env
env = DummyVecEnv([make_hardcore_train])
# Load previous stats
env = VecNormalize.load("./logs/improved/vec_normalize.pkl", env)
# WE WANT to update stats now because Hardcore data is different
env.training = True
env.norm_reward = True

# 2. Load the Pre-Trained Model
model = PPO.load("./logs/improved/final_model", env=env)

# 3. Lower the learning rate
# We don't want to destroy what it already learned, just adapt it.
model.learning_rate = 1e-4

# 4. Train on Hardcore
# This usually takes A LOT of steps (1 Million+), but we'll try 200k to see progress.
model.learn(total_timesteps=200_000)

model.save("./logs/hardcore_train/final_hardcore_model")
env.save("./logs/hardcore_train/vec_normalize.pkl")
env.close()
print("Hardcore Training Done! Now try running the visualization on this new model.")

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from gymnasium.wrappers import RecordVideo
import glob
import os
import shutil

# --- 1. CLEANUP (Fixes the "old video" confusion) ---
# We delete the folder so we know for sure the video we see is NEW.
if os.path.exists("./videos/hardcore"):
    shutil.rmtree("./videos/hardcore")
os.makedirs("./videos/hardcore", exist_ok=True)

# --- 2. SETUP ---
def make_hardcore():
    env = gym.make("BipedalWalker-v3", hardcore=True, render_mode="rgb_array")
    env = RecordVideo(
        env,
        video_folder="./videos/hardcore",
        name_prefix="hardcore_run",
        episode_trigger=lambda x: True # Record every episode
    )
    return env

hardcore_env = DummyVecEnv([make_hardcore])

# Load Normalization Stats
stats_path = "./logs/improved/vec_normalize.pkl"
if os.path.exists(stats_path):
    hardcore_env = VecNormalize.load(stats_path, hardcore_env)
    hardcore_env.training = False
    hardcore_env.norm_reward = False
else:
    print("⚠️ WARNING: No normalization stats found. The robot will likely fall instantly.")

# Load Model
model_path = "./logs/improved/final_model.zip"
model = PPO.load(model_path, env=hardcore_env)

# --- 3. RUN ---
print("--- 🏃 STARTING NEW HARDCORE RUN 🏃 ---")
obs = hardcore_env.reset()
done = False

# We limit to 1000 steps so the video saves even if it gets stuck
for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = hardcore_env.step(action)
    if done:
        break

hardcore_env.close()
print("--- Run Complete ---")

# --- 4. SHOW ---
video_files = glob.glob("./videos/hardcore/*.mp4")
if video_files:
    # Since we cleaned the folder, there should only be ONE video.
    print(f"Showing the ONLY video in folder: {video_files[0]}")
    show_video(video_files[0])
else:
    print("❌ No video found. The agent might have crashed before step 1.")

--- 🏃 STARTING NEW HARDCORE RUN 🏃 ---
--- Run Complete ---
Showing the ONLY video in folder: ./videos/hardcore/hardcore_run-episode-0.mp4


In [ ]:
import gymnasium as gym
import torch.nn as nn
import numpy as np
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor

# Directories
LOG_DIR = "./logs/improved/"
VIDEO_DIR = "./videos/improved/"
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

# --- 1. Custom Reward Wrapper ---
class StabilityRewardWrapper(gym.Wrapper):
    def __init__(self, env, upright_bonus=0.01):
        super().__init__(env)
        self.upright_bonus = upright_bonus

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        # obs[0] is hull angle. We want it close to 0.
        stability = self.upright_bonus * np.exp(-1 * (obs[0] ** 2))
        return obs, reward + stability, terminated, truncated, info

def make_env():
    env = gym.make("BipedalWalker-v3", render_mode="rgb_array")
    # Wrap to record video every 50 episodes, length 2000 steps
    env = gym.wrappers.RecordVideo(
        env,
        video_folder=VIDEO_DIR,
        episode_trigger=lambda x: x % 50 == 0,
        name_prefix="bipedal-agent",
        video_length=2000
    )
    env = StabilityRewardWrapper(env)
    env = Monitor(env, LOG_DIR)
    return env

# --- 2. Environment Setup ---
# Create environment
env = DummyVecEnv([make_env])

# CRITICAL: Normalize observations (-3.14 to +5.0 is too wide for raw input)
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

# --- 3. The Bigger "Brain" (Policy) ---
# We need a larger network to process the 24 inputs (Lidar + Joints)
policy_kwargs = dict(
    activation_fn=nn.Tanh,
    net_arch=dict(pi=[256, 256], vf=[256, 256])
)

model = PPO(
    "MlpPolicy",
    env,
    # --- The Brain ---
    # policy_kwargs=dict(
    #     activation_fn=nn.Tanh,
    #     net_arch=dict(pi=[256, 256], vf=[256, 256])
    # )
    policy_kwargs=policy_kwargs,

    # --- Learning Speed ---
    learning_rate=3e-4,        # Standard speed

    # --- Data Collection ---
    n_steps=2048,              # Steps per update
    batch_size=64,             # Minibatch size

    # --- The "Secret Sauce" ---
    # gamma=0.99,                # Discount factor (Farsightedness)
    # gae_lambda=0.95,           # Smoothing
    # clip_range=0.2,            # Safety limit for updates

    # --- Behavior ---
    ent_coef=0.005,            # Slight randomness to prevent getting stuck
    # vf_coef=0.5,               # Importance of the "Critic" (Value Function)
    # max_grad_norm=0.5,         # Prevent mathematical explosions

    verbose=1
)

# --- 4. Train ---
print("Starting Training (This may take 15-20 mins on Colab)...")
# Note: To truly "solve" it (300+ reward), you often need ~500k-1M steps.
# We run 200k here to show progress.
model.learn(total_timesteps=200_000)

# Save Model & Normalization Stats
model.save(f"{LOG_DIR}/final_model")
env.save(f"{LOG_DIR}/vec_normalize.pkl")
env.close()

print("Training Done!")

In [ ]:
import glob

# Find the latest video in the folder
video_files = glob.glob(f"{VIDEO_DIR}/*.mp4")
# Sort by modification time to get the newest one
video_files.sort(key=os.path.getmtime)

if video_files:
    latest_video = video_files[-1]
    print(f"Showing: {latest_video}")
    show_video(latest_video)
else:
    print("No videos found.")

Showing: ./videos/improved/bipedal-agent-episode-100.mp4


In [ ]:
# ... (Imports from previous cells) ...

# 1. Load the Hardcore Env (NO Wrapper if possible!)
def make_hardcore_train():
    # render_mode="rgb_array" is vital for video but optional for pure training speed
    env = gym.make("BipedalWalker-v3", hardcore=True, render_mode="rgb_array")
    env = Monitor(env, "./logs/hardcore_train/")
    return env

print("--- Starting Curriculum Learning (Normal -> Hardcore) ---")

# Setup Env
env = DummyVecEnv([make_hardcore_train])

# 2. Load Stats (The "Glasses")
# This is correct. We load the "Normal" vision but set training=True
# so it can adapt to seeing stumps and pits.
env = VecNormalize.load("./logs/improved/vec_normalize.pkl", env)
env.training = True
env.norm_reward = True

# 3. Load the Pre-Trained Model (The "Brain")
model = PPO.load("./logs/improved/final_model", env=env)

# --- 4. ADJUST PARAMETERS FOR HARDCORE ---

# A. Lower Learning Rate (Protect the walking knowledge)
model.learning_rate = 1e-4

# B. Increase Entropy (Encourage trying risky jumps)
# We have to set this manually because .load() overwrites it with the old value.
model.ent_coef = 0.01

# 5. Train on Hardcore
print("Starting Hardcore Training...")
# Realistically, you need millions of steps.
# But for a test, we will run 500k so you might at least see it TRY to jump.
model.learn(total_timesteps=200_000)

model.save("./logs/hardcore_train/final_hardcore_model")
env.save("./logs/hardcore_train/vec_normalize.pkl")
env.close()
print("Hardcore Training Done!")

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from gymnasium.wrappers import RecordVideo
import glob
import os
import shutil

# --- 1. CLEANUP (Fixes the "old video" confusion) ---
# We delete the folder so we know for sure the video we see is NEW.
if os.path.exists("./videos/hardcore"):
    shutil.rmtree("./videos/hardcore")
os.makedirs("./videos/hardcore", exist_ok=True)

# --- 2. SETUP ---
def make_hardcore():
    env = gym.make("BipedalWalker-v3", hardcore=True, render_mode="rgb_array")
    env = RecordVideo(
        env,
        video_folder="./videos/hardcore",
        name_prefix="hardcore_run",
        episode_trigger=lambda x: True # Record every episode
    )
    return env

hardcore_env = DummyVecEnv([make_hardcore])

# Load Normalization Stats
stats_path = "./logs/improved/vec_normalize.pkl"
if os.path.exists(stats_path):
    hardcore_env = VecNormalize.load(stats_path, hardcore_env)
    hardcore_env.training = False
    hardcore_env.norm_reward = False
else:
    print("⚠️ WARNING: No normalization stats found. The robot will likely fall instantly.")

# Load Model
model_path = "./logs/improved/final_model.zip"
model = PPO.load(model_path, env=hardcore_env)

# --- 3. RUN ---
print("--- 🏃 STARTING NEW HARDCORE RUN 🏃 ---")
obs = hardcore_env.reset()
done = False

# We limit to 1000 steps so the video saves even if it gets stuck
for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = hardcore_env.step(action)
    if done:
        break

hardcore_env.close()
print("--- Run Complete ---")

# --- 4. SHOW ---
video_files = glob.glob("./videos/hardcore/*.mp4")
if video_files:
    # Since we cleaned the folder, there should only be ONE video.
    print(f"Showing the ONLY video in folder: {video_files[0]}")
    show_video(video_files[0])
else:
    print("❌ No video found. The agent might have crashed before step 1.")

--- 🏃 STARTING NEW HARDCORE RUN 🏃 ---
--- Run Complete ---
Showing the ONLY video in folder: ./videos/hardcore/hardcore_run-episode-0.mp4
